In [ ]:
#upload workday's My_Recruiting_Jobs_-_More_Details report and transform

#import packages
import pandas as pd
import os
import io
import numpy as np

In [ ]:
from google.colab import files
uploaded_df = files.upload()
uploaded_glossary = files.upload()

Saving My_Recruiting_Jobs_-_More_Details.xlsx to My_Recruiting_Jobs_-_More_Details (3).xlsx


Saving Departments 2022.xlsx to Departments 2022 (2).xlsx


In [ ]:
df = pd.read_excel(io.BytesIO(uploaded_df['My_Recruiting_Jobs_-_More_Details.xlsx']))
df_glossary = pd.read_excel(io.BytesIO(uploaded_glossary['Departments 2022.xlsx']))

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#remove reqs with 0 openings
df = df.loc[df['Number of Openings Available'] != 0]

#remove reqs with 'freeze' or 'filled' in req name
df = df.loc[df['Job Requisition'].str.contains('Freeze Date')==False]
df = df.loc[df['Job Requisition'].str.contains('Fill Date')==False]

In [ ]:
#fix dtypes
# print(df.dtypes)

#remove extra text from start date col
df['Recruiting Start Date'] = df['Recruiting Start Date'].str[:10]
#change to date type
df['Recruiting Start Date'] = pd.to_datetime(df['Recruiting Start Date'], format='%m/%d/%Y')
df['Recruiting Start Date'] = df['Recruiting Start Date'].dt.date

In [ ]:
#remove extra text in job requisition column
df['Job Requisition'] = df['Job Requisition'].str[9:-7]

In [ ]:
df = df.drop(['Candidates in Review Stage', 'Active Candidates', 'Manager Photo', 'Supervisory Organization', 'Additional Locations',
         'Date Request Entered', 'Request Completed Date', 'Target Hire Date', 'Job Family',
         'Division', 'Cost Center Hierarchy'], axis=1)

In [ ]:
#add columns
df['Status'] = ''
df['Days Open'] = np.nan
df['Notes'] =''
df['Division'] = ''
df['Business Unit'] = ''

#reorganize column order
df = df[['Job Requisition', 'Status', 'Number of Openings Available', 'Recruiting Start Date', 'Days Open',
         'Notes', 'Recruiter', 'Division', 'Business Unit', 'Cost Center',
         'Primary Location', 'Hiring Manager', 'Create Job Requisition Reason', 'Replacement for Worker']]

#rename column names
df = df.rename(columns={'Job Requisition':'Job', 'Number of Openings Available':'Openings', 'Recruiting Start Date':'Open Date', 
                        'Cost Center':'Department', 'Create Job Requisition Reason':'Reason for Hire', 
                        })

In [ ]:
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
  match_value = return_array.loc[lookup_array == lookup_value]
  if match_value.empty:
    return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found
  else:
    return match_value.tolist()[0]

In [ ]:
# xlookup division, bu columns
df['Business Unit'] = df['Department'].apply(xlookup, args = (df_glossary['Department'], df_glossary['Business Unit']))
df['Division'] = df['Department'].apply(xlookup, args = (df_glossary['Department'], df_glossary['Division']))

In [ ]:
#correct reason for hire text
df['Reason for Hire'].mask(df['Reason for Hire'] == 'Recruiting > Backfill', 'Backfill', inplace=True)
df['Reason for Hire'].mask(df['Reason for Hire'] == 'Recruiting > New Hire', 'New Hire', inplace=True)

In [ ]:
#sort
df.sort_values(['Recruiter', 'Division', 'Business Unit', 'Department'], inplace=True)

In [ ]:
df.to_excel('Jobs Summary WD.xlsx', encoding = 'utf-8-sig', index=False) 
files.download('Jobs Summary WD.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>